!jt -t chesterish
# !jt -r

In [1]:
from genetic_algorithm import *
from movement import *
from numpy.random import default_rng
import pickle

import binascii


import matplotlib.pyplot as plt
import math
from matplotlib.animation import FuncAnimation
# %matplotlib notebook

import copy
from collections import Counter
import pandas as pd

In [2]:
## hexadecimal generator

nr_of_input = 2
nr_of_actions = 5
nr_of_inner = 2
nr_of_genes = 8
nr_individuals = 60

nr_of_generations = 10

## world size
world_size = 10
world_size_x = world_size
world_size_y = world_size

## generations

In [3]:
# for gen_nr in range(nr_of_generations):
#     print(gen_nr)
#     if gen_nr == 0:
#         result = initial_population(nr_individuals, nr_of_genes, nr_of_input, nr_of_actions, nr_of_inner, world_size)
#         result, a,b,c,d,e = steps_in_generation(world_size*1.2, result, world_size_x, world_size_y)
#         print(a,b,c,d,e)

time_list = []
result = {}
for gen_nr in range(nr_of_generations):
    print(gen_nr)
    if gen_nr == 0:
        result = initial_population(nr_individuals, nr_of_genes, nr_of_input, nr_of_actions, nr_of_inner, world_size)
        result = steps_in_generation(world_size*1.2, result, world_size_x, world_size_y)
    else:
        
        result = asexual_reproduction_and_mutation(world_size, result, nr_individuals)
        if result:
            result = next_generation(result,nr_of_input, nr_of_actions, nr_of_inner, world_size, nr_individuals)
            result = steps_in_generation(world_size*1.2, result, world_size_x, world_size_y)
    result.update(result)
    
    dic_color = {}
    for indiv in result:
        rgb_colors = list(map(hex_to_rgb, result[indiv]['genome']))
        color = tuple(pd.DataFrame(rgb_colors).median()) 
        dic_color[indiv] = color
        
    coords = generate_dictionary_of_coords(result, world_size*2, dic_color)
    
    def update(i):
        ax.clear()
        ax.set_facecolor(plt.cm.Blues(.2))

        ax.set_xlim([0,world_size_x])
        ax.set_ylim([0,world_size_y])
        ax.set_title('moving')
        ax.scatter(x=coords[i]['x'],y=coords[i]['y'], c=coords[i]['color'], s=20, marker='o')
        [spine.set_visible(False) for spine in ax.spines.values()]


    fig, ax = plt.subplots(figsize=(6,6))
    myAnimation = FuncAnimation(
        fig = fig,
        func = update,
        frames = len(coords),
        interval = 10, repeat=False
    )
    myAnimation.save(f'./output/generation-{gen_nr}.gif', writer='imagemagick' , fps=10)
    # create a binary pickle file 
    f = open(f'./output/generation-{gen_nr}.pkl',"wb")
    pickle.dump(result, f)
    f.close()

In [4]:
# def steps_in_generation(world_size, result, world_size_x, world_size_y):
#     for step_nr in range(int(world_size)):
#         for indiv in result:
#             position = result[indiv]['position']
#             x, y = position[-1][0], position[-1][1]
#             if step_nr < 1:
#                 calculate_position(result, indiv, x, y, world_size_x, world_size_y)
#             elif step_nr >= 1:
#                 apply_input(result, indiv)
#                 calculate_position(result, indiv, x, y, world_size_x, world_size_y)

#             # prevent_overlap_movement(last_pos_list, result)
#             for prev in sorted(result.keys()): 
#                 if prev != indiv and result[prev]['position'][-1] == position[-1]:
#                     position[-1] = position[-2]


In [5]:
for gen_nr in range(nr_of_generations):
    print(gen_nr)
    if gen_nr == 0:
        s = time.time()
        result = initial_population(nr_individuals, nr_of_genes, nr_of_input, nr_of_actions, nr_of_inner, world_size)
        steps_in_generation(world_size*1.2, result, world_size_x, world_size_y)
        
        e = time.time()
    else:
        s = time.time()
        result = asexual_reproduction_and_mutation(world_size, result, nr_individuals)
        if result:
            result = next_generation(result,nr_of_input, nr_of_actions, nr_of_inner, world_size, nr_individuals)
            steps_in_generation(world_size*1.2, result, world_size_x, world_size_y)
        e = time.time()
    print(e-s)
    print(result[0]['position'])

0
0.009048700332641602
[[3, 6]]
1
0.07504868507385254
[[0, 2]]
2
0.045758962631225586
[[3, 8]]
3
0.05718111991882324
[[9, 1]]
4
0.05295562744140625
[[1, 0]]
5
0.04982590675354004
[[2, 1]]
6
0.03986167907714844
[[3, 5]]
7
0.0400543212890625
[[6, 3]]
8
0.04531216621398926
[[2, 3]]
9
0.03487396240234375
[[4, 6]]


In [5]:
result[0]

{'out': {'out0': 0.525, 'out2': 0.97, 'out4': 0.507},
 'brain': {'mid0out0': ['mid0', 'out0', 0.0494430032046391],
  'mid0mid0': ['mid0', 'mid0', -3.700289943537311],
  'in0out2': ['in0', 'out2', 1.7461315428048223],
  'in0mid1': ['in0', 'mid1', 2.9750038150465437],
  'mid0out2': ['mid0', 'out2', -3.116984587211964],
  'mid0out4': ['mid0', 'out4', -1.117656035403632]},
 'in': ['in0'],
 'genome': ['d2c1bb7a',
  'ec6c5ced',
  '5eaab7df',
  '3c5ddf31',
  'b6f88195',
  '825a7666',
  'd4cd63bc',
  'baf223c3'],
 'position': [[69, 9]]}

In [4]:
def steps_in_generation(world_size, result, world_size_x, world_size_y):
    n = 0
    while  world_size>n:
        for indiv in result:
            x, y = result[indiv]['position'][-1][0], result[indiv]['position'][-1][1]
            if n<1:
                calculate_position(result, indiv, x, y, world_size_x, world_size_y)
            elif n >= 1:
                apply_input(result, indiv)
                calculate_position(result, indiv, x, y, world_size_x, world_size_y)
            # prevent_overlap_movement(last_pos_list, result)
            for prev in sorted(result.keys()): 
                if prev != indiv and result[prev]['position'][-1] == result[indiv]['position'][-1]:
                    result[indiv]['position'][-1] = result[indiv]['position'][-2]
        n += 1
    yield result

In [5]:
rr = steps_in_generation(world_size, result, world_size_x, world_size_y)